# Contract Analysis Using 6B Model on Laptop with LLMWare

### Introduction

Imagine you were given a large contract and asked a really specific question about it: "What is the notice for termination for convenience?" It would be an ordeal to locate the answer for this in the contract.

But what if we could use AI 🤖 to analyze the contract and answer this for us?

What we want here is to perform something known as retrieval-augmented generation (RAG). This is the process by which we give a language model some external sources (such as a contract). The external sources are intended to enhance the model's context, giving it a more comprehensive understanding of a topic. The model should then give us more accurate responses to the questions we ask it on the topic.

Now, a general purpose model like Chat-GPT might be able to answer questions about contracts with RAG, but we can do something better. We can use LLMWare's dragon-yi-6b-gguf model, which is RAG-finetuned for fact-based question-answering on complex business and legal documents.

### For Google Colab Users

If you are using Colab for free, we highly recommend you activate the T4 GPU hardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as apposed to the ~13GB Colab gives automatically.

To activate T4:
1. click on the "Runtime" tab
2. click on "Change runtime type"
3. select T4 GPU under Hardware Accelerator

NOTE: there is a weekly usage limit on using T4 for free

### Installing and importing dependencies

In [6]:
%pip install llmware

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os

from llmware.setup import Setup
from llmware.library import Library
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.retrieval import Query
from llmware.configs import LLMWareConfig

### MongoDB

This example requires you to have an instance of MongoDB running to store the processed contracts. The easiest way to do this is to use Docker Compose with our provided file `llmware/docker-compose_mongo_milvus.yaml`. More information on how to do this is available in our [documentation](https://github.com/llmware-ai/llmware/blob/main/docs/getting_started/working_with_docker.md).

### Worker function

In this function, we will do the following:
1. Loading in contracts using the `Setup` class
2. Adding them to a library using the `Library` class
3. Locate MSA files using a text query on the first page of each document using the `Query` class
4. Perform a text query to find all text containing "termination"
5. Pass these text chunks as a source to our _dragon-yi-6b-gguf_ model with the `Prompt` class
6. Ask the model the question "What is the notice for termination for convenience?"
7. Output the model's response
8. Save the model's state as a CSV with the `HumanInTheLoop` class.

In [8]:
def msa_processing():

    local_path = Setup().load_sample_files()
    agreements_path = os.path.join(local_path, "AgreementsLarge")

    #   create a library with all of the Agreements (~80 contracts)
    msa_lib = Library().create_new_library("msa_lib503_635")
    msa_lib.add_files(agreements_path)

    #   find the "master service agreements" (MSA)
    q = Query(msa_lib)
    query = "master services agreement"
    results = q.text_search_by_page(query, page_num=1, results_only=False)

    #   results_only = False will return a dictionary with 4 keys:  {"query", "results", "doc_ID", "file_source"}
    msa_docs = results["file_source"]

    #   load prompt/llm locally
    model_name = "llmware/dragon-yi-6b-gguf"
    prompter = Prompt().load_model(model_name)

    #   analyze each MSA - "query" & "llm prompt"
    for i, docs in enumerate(msa_docs):

        print("\n")
        print (i+1, "Reviewing MSA - ", docs)

        #   look for the termination provisions in each document
        doc_filter = {"file_source": [docs]}
        termination_provisions = q.text_query_with_document_filter("termination", doc_filter)

        #   package the provisions as a source to a prompt
        sources = prompter.add_source_query_results(termination_provisions)

        print("update: sources - ", sources)

        #   call the LLM and ask our question
        response = prompter.prompt_with_source("What is the notice for termination for convenience?")

        #   post processing fact checking
        stats = prompter.evidence_comparison_stats(response)
        ev_source = prompter.evidence_check_sources(response)

        for i, resp in enumerate(response):
            print("update: llm response - ", resp)
            print("update: compare with evidence- ", stats[i]["comparison_stats"])
            print("update: sources - ", ev_source[i]["source_review"])

        prompter.clear_source_materials()

    # Save jsonl report with full transaction history to /prompt_history folder
    print("\nupdate: prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))

    prompter.save_state()

    # Generate CSV report for easy Human review in Excel
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()

    print("\nupdate: csv output for human review - ", csv_output)

    return 0

### Main block

Here, we call our worker function.

In [9]:
if __name__ == "__main__":

    m = msa_processing()



1 Reviewing MSA -  C:\Users\prash\llmware_data\tmp\parser_tmp\process_pdf_files\Testco MSA Delta.pdf
update: sources -  {'text_batch': ["must be   brought within two years from the date that the cause of action arose.   Term and Termination of Agreement; Assignment.   This Agreement shall remain in effect until terminated. Either party may terminate this agreement,  any Statement of   Work or Services Description for convenience by giving the other party 30 days   written  notice.  Either  party  may   terminate  this  Agreement  or \nmust be   brought within two years from the date that the cause of action arose.   Term and Termination of Agreement; Assignment.   This Agreement shall remain in effect until terminated. Either party may terminate this agreement,  any Statement of   Work or Services Description for convenience by giving the other party 30 days   written  notice.  Either  party  may   terminate  this  Agreement  or \n party  may  be   deemed  notice  of   termination  o

Our output contains a dictionary for each LLM response with several keys, notably:
- `llm_response`: giving us the answer to our question
- `evidence`: giving us the text where the model found the answer to the question